# Model drift

In [4]:
from sklearn.cluster import MiniBatchKMeans
import joblib

# N là số lượng cluster
N = 19000 * len(np.unique(train_y))
# Train clustering model cho data đã có label
#kmeans = MiniBatchKMeans(n_clusters=N, random_state=0, n_init='auto').fit(train_x)

#joblib.dump(kmeans, 'data/captured_data/phase-1/prob-1/kmeans.cpk')
kmeans = joblib.load('data/captured_data/phase-1/prob-1/kmeans.cpk')

# Tạo 1 mảng ánh xạ cluster với 1 label mới (do các data drift thuộc cùng 1 cluster sẽ có label giống nhau)
new_labels = []

# Duyệt từng cluster
for  i  in  range(N):
	# Lấy các label của các data point thuộc cluster i
	mask = (kmeans.labels_ == i)
	cluster_labels = train_y[mask]

	if  len(cluster_labels) == 0:
		# Nếu cluster i rỗng thì xác định cluster i ánh xạ với 1 label mặc định (ở đây lựa chọn là 0)
		new_labels.append(0)
	else:
		# Tìm label mới cho cả cụm cluster trong trường hợp cụm cluster khác rỗng
		#if  isinstance(train_y.flatten()[0], float):
			# Nếu là bài toán Regression thì lấy giá trị trung bình của các label thuộc cluster
		#	new_labels.append(np.mean(cluster_labels.flatten()))
		#else:
			# Nếu là bài toán Classification thì lấy label xuất hiện nhiều nhất trong cluster
			new_labels.append(np.bincount(cluster_labels.to_numpy().flatten()).argmax())

# Ánh xạ lại label cho data drift dựa trên kết quả predict cluster ở trên
y_drift_propagated = [new_labels[c] for  c  in  kmeans.labels_]

In [5]:
from sklearn.metrics import roc_auc_score
y_drift_test_propagated = [ new_labels[c] for c in kmeans.predict(train_x)]
propagated_auc = {"test_auc": roc_auc_score(train_y, y_drift_propagated)}
propagated_auc

{'test_auc': 0.8626996177558589}

# Online Data

In [61]:
import pandas as pd
import redis
import pickle
# Load data for problem 1
rc1 = redis.Redis(host='192.168.88.113', db=1, port=6379,  socket_keepalive=True)

captured_x = pd.DataFrame()
for key in rc1.keys():
    captured_data = pickle.loads(rc1.get(key))
    captured_x = pd.concat([captured_x, captured_data])

#captured_x.drop_duplicates(inplace=True, ignore_index=True)
#captured_x = apply_category_features(
#    raw_df=captured_x[train_x.columns],
#    categorical_cols=prob_config.categorical_cols,
#    category_index=category_index,
#)

In [62]:
allkey = rc1.keys()

In [13]:
rc1test= redis.Redis(host='localhost', db=1, port=6379,  socket_keepalive=True)

# Test API

In [182]:
rows = []
test = pickle.loads(rc1.get(allkey[8]))

for index, row in test.iterrows():
    rows.append(row.to_list())

data = {
  "id": "123",
  "rows": rows,
  "columns": test.columns.to_list()
}

import requests
#response = requests.post('http://14.225.205.204:80/phase-2/prob-1/predict', json=data)
#response = requests.post('http://27.71.25.203:5040/phase-2/prob-1/predict', json=data)
response = requests.post('http://localhost:8000/phase-2/prob-1/predict', json=data)
print(response.text)

{"id":"123","predictions":[0.00012732274553229182,0.9982673285285959,0.9998837423846397,0.9998837423846397,0.5198779988175567,0.9996763701451702,0.18417014259410575,0.9900059583606529,6.500185083784603e-05,0.002642677417957026,0.999905894469594,0.9998837423846397,0.43133762025428024,0.9991340247045126,0.9998688087824612,0.9993121197912591,0.9996805919320815,0.999905894469594,0.0001235460814705884,0.0003030946270267483,0.00015151533312950143,0.9973949644942843,0.9994136686467642,0.9998837423846397,0.9990595520826252,0.9991831517009513,0.00013271927225128028,0.23101838015191478,0.9998837423846397,0.9996828965823179,0.9917356969439712,0.0001439515349922888,0.9998837423846397,0.999905894469594,0.9996808243884079,0.9981453480286913,0.00018260885479962655,0.5727286715738497,0.9901553190976862,0.00014967490737515443,0.9998837423846397,0.00016502391246864254,0.9995868099469248,0.1728174416857805,0.9990215332377335,0.9958592441098814,0.9984596352613586,0.9998837423846397,0.00013615017190342937,

In [185]:
for i in range(10):
    response = requests.post('http://localhost:8000/phase-2/prob-1/predict', json=data)

In [34]:
from aiocache import Cache
from aiocache.serializers import PickleSerializer

cacherequest = Cache(Cache.REDIS, endpoint="localhost", port=6379, db=1, serializer=PickleSerializer())
await cacherequest.clear()

True

In [28]:
import paho.mqtt.client as mqtt
mqtt_topic = f'logging/phase-2/prob-1'
mqtt_cient = mqtt.Client()
try:
    mqtt_cient.connect('127.0.0.1', port=1883)
    print("Sucess")
    mqtt_cient.loop_start()
except:
    print("Falied")

Sucess


In [200]:
import orjson
import random
import datetime
mqtt_topic = f'logging/phase-2/prob-2'
for i in range(100):
    current_dt = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S")

    runtime = random.randint(3, 50)
    result = {
        "request_time": current_dt,
        "runtime": runtime
    }

    (rc, mid) = mqtt_cient.publish(mqtt_topic, orjson.dumps(result))

In [ ]:
mqtt_cient.loop_stop